In [1]:
# Breakdown sentiment by ChatGPT and human
# look at themes and topics of the 30 and 40 slections. Maybe too much emotion or common pattern
# Focus on the domain in which llm or human wins.
# look at complexitty of tweets and
# looka at null frames

###########################################
# Analyzing the number of human responses #
# vs ChatGPT responses selected.          #
###########################################

import json

chatgpt_selected = 0
human_selected = 0

i=0

input_messages: list = []

gpt_selected_msgs: list = []
human_selected_msgs: list = []

# A list of tweets which yielded atleast 1 null answer choice.
null_prompts = []

# Open your NDJSON file
with open('/humanpref.ndjson', 'r') as file:
    for line in file:
        try:
            data_dict = json.loads(line)  # Parse each line as JSON

            row_data_str = data_dict.get('data_row', {}).get('row_data', '')

            if row_data_str:
                # Parse the stringified JSON in 'row_data'
                row_data_dict = json.loads(row_data_str)

                # Now extract modelOutputs from the parsed row_data
                model_outputs = row_data_dict['modelOutputs']

                # Save the social media posts for later
                input_messages.append(row_data_dict.get('messages')[0]['content'])

                # Identify responses belonging to chatGPT4o and others
                chatgpt4o_response = None
                other_response = None

                for output in model_outputs:
                    if output.get('modelConfigName') == 'ChatGPT4o':
                        chatgpt4o_response = output
                    else:
                        other_response = output

                    if output.get('content') == "None" and (row_data_dict.get('messages')[0]['content']) not in null_prompts:
                        null_prompts.append( row_data_dict.get('messages')[0]['content'] )


                # If we have both a chatGPT4o response and another response
                if chatgpt4o_response and other_response:
                    # Extract the radio answer
                    id = ""
                    for x in data_dict['projects']:
                        id = x
                        break

                    i+=1

                    classi = data_dict['projects'][id]['labels']

                    if len(classi) > 0:
                        x = classi[0]['annotations']['classifications']

                        if len(x) > 0:
                            radio_answer = x[0].get('radio_answer')

                            # Check if the "name" or "value" matches the chatGPT4o response
                            is_name_match = radio_answer.get('name') == chatgpt4o_response.get('title')[-1]
                            is_value_match = radio_answer.get('value') == chatgpt4o_response.get('title')[-1]

                            if radio_answer.get('name') == chatgpt4o_response.get('title')[-1]:
                                chatgpt_selected += 1
                                gpt_selected_msgs.append(row_data_dict.get('messages')[0]['content'])
                            if radio_answer.get('name') == other_response.get('title')[-1]:
                                human_selected_msgs.append(row_data_dict.get('messages')[0]['content'])
                                human_selected += 1

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

print("ChatGPT selected ", chatgpt_selected)
print("Human selected ", human_selected)

print(input_messages)


"""OPTIONAL: Truncate the Null Prompts to ignore blanks. """
#null_prompts = [x for x in null_prompts if len(x) > 4]

ChatGPT selected  30
Human selected  42
['RT @BucksCoBeacon: Newtown Community Shuts Down Hateful Propaganda Movie Screening | Right-wing parents wanted to show a doc that claims public school teachers teach kids “X-rated sex acts” &amp; pushes other conspiracies often heard at #BucksCounty school board meetings. READ: https://t.co/7sdex6iVAD https://t.co/tCgD4xMZwj', 'RT @JeremiahSladeck: “Chronology is an enemy of autocracy” might be one of the best sentences ever written. https://t.co/AHTeNwVxxm', 'Trump 2024? 60% of Voters Say ‘No’DeSantis favorability beats Trump significantly in every demographic, especially when comparing Very Unfavorables.\r\n\r\nBut Republican and Independent voters are only 4 points more likely to say DeSantis should run than Trump.\r\n\r\nhttp://bit.ly/3tBtWqm', 'Voters Still Concerned About High Gas PricesVery Concerned about gas prices:\r\n\r\nAll - 57%\r\nMen - 55%\r\nWomen - 59%\r\nWhite - 60%\r\nBlack - 46%\r\nHisp/oth - 55%\r\nRep - 73%\r\nDem - 40%\r\

In [5]:
###########################################
# Analyzing the sentiment of each input.  #
#                                         #
###########################################

# Add more than positive and negative

from transformers import pipeline

results: list = []

# Initialize the pipeline
sentiment_analyzer = pipeline('sentiment-analysis')

# Input for analysis
for text in input_messages:
    # Perform sentiment analysis

    result = sentiment_analyzer(text)

    results.append(result[0]['label'])

positive_results = results.count("POSITIVE")
negative_results = results.count("NEGATIVE")

In [3]:
###########################################
# Extract emotional sentiments from the   #
# tweets which go beyond + and - .        #
###########################################

from collections import Counter

print(human_selected_msgs)
print(gpt_selected_msgs)

# Load a pre-trained emotion classification model
emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

# Analyze emotions in a text
# text = "I'm so excited about my new job, but a bit nervous too."

# Initialize a counter to tally the emotions
gpt_emotion_counts = Counter()
human_emotion_counts = Counter()

# Loop through each text and get the prevailing emotion
for text in gpt_selected_msgs:
    result = emotion_pipeline(text)
    # Find the emotion with the highest score
    prevailing_emotion = max(result[0], key=lambda x: x['score'])
    # Increment the count for the prevailing emotion
    gpt_emotion_counts[prevailing_emotion['label']] += 1

# Loop through each text and get the prevailing emotion
for text2 in human_selected_msgs:
    result = emotion_pipeline(text2)
    # Find the emotion with the highest score
    prevailing_emotion = max(result[0], key=lambda x: x['score'])
    # Increment the count for the prevailing emotion
    human_emotion_counts[prevailing_emotion['label']] += 1

# Output the tally of emotions
print("GPT emotion counts:")
for emotion, count in sorted(gpt_emotion_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{emotion}: {count}")

# Output the tally of emotions
print("\nHuman emotion counts:")
for emotion, count in sorted(human_emotion_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{emotion}: {count}")



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: Failed to import transformers.models.distilbert.modeling_tf_distilbert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
#######################################################################
# Calculate average TTR of datasets to determine lexical complexity.  #
# Calculate the readabiliity of the dataset and compare them.         #
#######################################################################

import re

def syllable_count(word):
    word = word.lower()
    vowels = "aeiouy"
    syllables = re.findall(r'[aeiouy]+', word)
    count = len(syllables)

    if word.endswith("e"):
      count -= 1

    if word.endswith("le") and len(word) > 2 and word[-3] not in vowels:
      count += 1

    if count <= 0:
      count = 1
    return count


from nltk import FreqDist
from nltk.tokenize import word_tokenize
import nltk
from nltk.tokenize import sent_tokenize


nltk.download('punkt')
humanTTRmean = 0
gptTTRmean = 0
hf = 0
gf = 0

for text in human_selected_msgs:
    tokens = word_tokenize(text)
    unique_tokens = set(tokens)
    ttr = len(unique_tokens) / len(tokens)
    humanTTRmean += ttr

    sentences = sent_tokenize(text)

    total_words = len(tokens)
    total_sentences = len(sentences)
    total_syllables = sum([syllable_count(word) for word in tokens])

    hf += 206.835 - (1.015 * (total_words / total_sentences)) - (84.6 * (total_syllables / total_words))

for text in gpt_selected_msgs:
    tokens = word_tokenize(text)
    unique_tokens = set(tokens)
    ttr = len(unique_tokens) / len(tokens)
    gptTTRmean += ttr

    sentences = sent_tokenize(text)

    total_words = len(tokens)
    total_sentences = len(sentences)
    total_syllables = sum([syllable_count(word) for word in tokens])

    gf += 206.835 - (1.015 * (total_words / total_sentences)) - (84.6 * (total_syllables / total_words))

humanTTRmean = humanTTRmean / len(human_selected_msgs)
gptTTRmean = gptTTRmean / len(gpt_selected_msgs)

# Type-Token Ratio (TTR): This metric measures the ratio of unique words (types)
# to the total number of words (tokens) in a text, providing an
# indication of vocabulary variety.
print("Human TTR mean: ", humanTTRmean)
print("GPT TTR mean: ", gptTTRmean, "\n")

# Flesch Readabilty score, higher the better.
print("Human Flesch score: ", hf/len(human_selected_msgs))
print("GPT Flesch score: ", gf/len(gpt_selected_msgs))

nf = 0
for text in null_prompts:
    tokens = word_tokenize(text)
    unique_tokens = set(tokens)
    ttr = len(unique_tokens) / len(tokens)
    gptTTRmean += ttr

    sentences = sent_tokenize(text)

    total_words = len(tokens)
    total_sentences = len(sentences)
    total_syllables = sum([syllable_count(word) for word in tokens])

    nf += 206.835 - (1.015 * (total_words / total_sentences)) - (84.6 * (total_syllables / total_words))

print("Null Flesch score: ", nf/len(null_prompts), "\n")




In [ ]:
average_length = lambda lst: sum(len(entry) for entry in lst) / len(lst) if lst else 0
print("Average Length of null prompts:", average_length(null_prompts))
print("Average Length of human selected msgs:", average_length(human_selected_msgs))
print("Average Length of gpt selected msgs:", average_length(gpt_selected_msgs))

print("\nTweets which resulted in one or more selections being \"None\".")
print(null_prompts)